<div style="text-align:center;"><img src="http://www.mf-data-science.fr/images/projects/intro.jpg" style='width:100%; margin-left: auto; margin-right: auto; display: block;' /></div>

# <font color="#641E16">Contexte</font>
Nous allons ici développer un algorithme de Machine Learning destiné à assigner automatiquement plusieurs tags pertinents à une question posée sur le célébre site Stack overflow.     
Ce programme s'adresse principalement aux nouveaux utilisateurs, afin de leur suggérer quelques tags relatifs à la question qu'ils souhaitent poser.

### Les données sources
Les données ont été captées via l'outil d’export de données ***stackexchange explorer***, qui recense un grand nombre de données authentiques de la plateforme d’entraide.     
Elles portent sur la période 2009 / 2020 et **uniquement sur les posts "de qualité"** ayant au minimum 1 réponse, 5 commentaires, 20 vues et un score supérieur à 5.

### Objectif de ce Notebook
Dans ce Notebook, nous allons traiter la partie **data cleaning et exploration des données**. Un second notebook traitera ensuite les approches supervisées et non supervisées pour traiter la création de Tags à partir des données textuelles.     

Tous les Notebooks du projet seront **versionnés dans Kaggle mais également dans un repo GitHub** disponible à l'adresse https://github.com/MikaData57/Analyses-donnees-textuelles-Stackoverflow

# <font color="#641E16">Sommaire</font>
1. [Importation et description des données](#section_1)

In [ ]:
# Install package for PEP8 verification
!pip install pycodestyle
!pip install --index-url https://test.pypi.org/simple/ nbpep8

In [ ]:
# Import Python libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer

# Library for PEP8 standard
from nbpep8.nbpep8 import pep8

## <font color="#641E16" id="section_1">Importation et description des données</font>

In [ ]:
# Define path to data
path = '../input/stackoverflow-questions-filtered-2011-2021/'

# Concat all CSV datasets in one Pandas DataFrame
df_columns = pd.read_csv(path+'StackOverflow_questions_2009.csv').columns
data = pd.DataFrame(columns=df_columns)
for f in os.listdir(path):
    temp = pd.read_csv(path+f)
    data = pd.concat([data, temp], axis=0)
data.head(3)

In [ ]:
# Print full dataset infos
data.info()

In [ ]:
# Describe data
data.describe()

Le jeu de données ne compte pas de valeurs nulles. Nous allons à présent vérifier la **longeur des différents titres** de la base :

In [ ]:
fig = plt.figure(figsize=(20, 12))
ax = sns.countplot(x=data.Title.str.len())
start, end = ax.get_xlim()
ax.xaxis.set_ticks(np.arange(0, end, 5))
plt.axvline(data.Title.str.len().median() - data.Title.str.len().min(),
            color="r", linestyle='--',
            label="Title Lenght median : "+str(data.Title.str.len().median()))
ax.set_xlabel("Lenght of title")
plt.title("Title lenght of Stackoverflow questions",
          fontsize=18, color="#641E16")
plt.legend()
plt.show()

Nous allons également ploter la répartition des **longueurs de la variable** `Body` *(les corps de texte des questions)*. L'étendue étant très importante, nous allons dans un premier temps **discrétiser ces longueur** pour ne pas surcharger les temps de calculs de projection graphique :

In [ ]:
# Discretizer for Body characters lenght
X = pd.DataFrame(data.Body.str.len())

# Sklearn discretizer with 200 bins
discretizer = KBinsDiscretizer(n_bins=200,
                               encode='ordinal',
                               strategy='uniform')
body_lenght = discretizer.fit_transform(X)
body_lenght = discretizer.inverse_transform(body_lenght)
body_lenght = pd.Series(body_lenght.reshape(-1))

In [ ]:
fig = plt.figure(figsize=(20, 12))
ax = sns.countplot(x=body_lenght)
start, end = ax.get_xlim()
ax.xaxis.set_ticks(np.arange(0, end, 25))
ax.set_xlabel("Lenght of Body (after discretization)")
plt.title("Body lenght of Stackoverflow questions",
          fontsize=18, color="#641E16")
plt.legend()
plt.show()

On remarque que la majeur partie des questions compte moins de 4000 caractères *(balises HTML compris)* mais certains posts dépassent les 31 000 caractères. 

In [ ]:
pep8(_ih)